# Example: Invert multiple Sentinel scenes

In this example, several Sentinel-2 scenes are inverted for two crops types (winter wheat and maize) using five agricultural field plots in Southern Germany for the year 2017. The necessary input data is provided in the data folder and consists of

- a ESRI shapefile containing the field boundaries as well as their land cover
- a list of cloud-free acquisition dates in 2017, compiled from Google Earth Engine (query see below)
- a config file for ProSAIL and the land cover classes that you should copy to your **OBIA4RTM** home directory

Using this configurations and data you will be able to derive crop-type specific parameters for 13 acquisition dates.

First, we will setup the necessary environment by important the **OBIA4RTM.processing_api** and importing the datasets. Make sure to have copied the content of the *../data/OBIA4RTM_CFG* folder to your OBIA4RTM home directory.

You can find out the location of the OBIA4RTM home directory by typing:

In [1]:
from OBIA4RTM.processing_api import API
api = API(use_gee=True)
api.get_OBIA4RTM_home()

The OBIA4RTM home is: '/home/lukas/OBIA4RTM'


## Preprocessing

After having things ready, we can start with the **pre-processing**.
In this example, we will use Google-Earth-Engine (make sure to have a valid account and a working Python API) for doing the atmospheric correction using the 6S model. First, we need to important the data into Python.

In [5]:
from OBIA4RTM.processing_api import API
import pandas as pd


# Google Earth Engine will be used for pre-processing the Sentinel-2 imagery
use_gee = True
# create a new API instance to handle the processing
api = API(use_gee)
# set the tablenames accordingly to allow OBIA4RTM to store the results in the
# PostgreSQL/PostGIS backend database
api.set_tablenames()

# setup the parameters controlling the preprocessing:
# this includes the acquisition dates as well as the geometry of the field
# parcels for which the inversion should be conducted and the way the
# atmospherical correction algorithm should be carried out

# the geometry of the bounding box in which the field parcels are located
# in lat/lon coordinates in decimal degrees
geom = [[11.652010083396522,48.24900906966563],
        [11.745054408435749,48.24667656534854],
        [11.748327321488217,48.31192435113173],
        [11.655723673233638,48.314324706548085],
        [11.652010083396522,48.24900906966563]]

# the acquisition dates were previously determined using Google Earth Engine
# searching for available Sentinel-2 scenes in 2017 covering the vegetation
# period which had a cloud coverage less than 20 percent
# the acquisition dates are stored in a ASCII-file and are read from it
filepath = r'../data/Acqui_Dates/acquisition_dates'
df = pd.read_csv(filepath, sep=":", header=None)
acqui_dates = df[[1]] # extract the acquisition dates

# Shapefile with the field parcel boundaries
shp = r'..data/SHP/2017_Multiply_Sample_Area.shp'

## Atmospheric Correction

If this yields no error message, then everything is ready to start the atmospheric correction to bring the data into L2 level and mask out clouds and cloud shadows. Moreover, the spectra will be extracted at the object level and stored for each acquisition date in the OBIA4RTM backend database:

In [ ]:
# Option for cloud masking and shadow detection after atmospherical correction
# -> use method provided by Sam Murphy (Option = 2)
option = 2

# empty dict for storing the retrieved scene ids from the imagery
mapping = dict()

# loop over the acquisition dates to pre-process the images and extract the
# spectra
for ii in range(acqui_dates.shape[0]):
    # get the current acquisition date ('YYYY-MM-DD')
    acqui_date = acqui_dates.iloc[ii].values[0].strip()
    # call gee_preprocessing -> returns scene_id from Sentinel-2 imagery
    scene_id = api.do_gee_preprocessing(geom,
                                        acqui_date,
                                        option,
                                        shp)
    mapping.update({scene_id : acqui_date})

## Inversion

After this snippet is finished (remember to have a constant internet connection), you can invert the processed scenes. Please note that this can take a couple of a hours as the databases *INSERTS* are relatively time-consuming in the current implementation of the programme...

In [ ]:
# use the best 10 solutions (mean)
num_best_solutions = 10
# do the inversion for the LUT classes 0 (bare soil) 1 (maize silage) and 2 (winter wheat)
luc_classes = [0, 1]
for ii in range(9, df.shape[0]): #acqui_dates.shape[0]):
    # get scene id
    scene_id = next(iter( mapping.items() ))[0]
    # get an instance of the inversion interface provided by processing API
    # and run it; return the inverted spectra
    status = api.do_inversion(scene_id, num_best_solutions, luc_classes,
                     return_specs=True)